## *Jeu de données IRVE*
-------------------
# Analyse des relations entre champs (intégrité des données)
------------------------------
## Contexte
- clarification du rôle des modèles de données dans les jeux de données (cf mise à jour récente des [guides data.gouv](https://guides.etalab.gouv.fr/qualite/documenter-les-donnees/))
- intégration d'une propriété "relationship" dans les schémas de données ([issue TableSchema](https://github.com/frictionlessdata/specs/issues/803) en cours de validation)
- création d'outils de contrôle des relations entre champs des jeux de données tabulaires (cf usage ci-dessous)

## Objectifs
- valider sur un cas réel l'utilisation d'un modèle de données en complément d'un schéma de données
- identifier les apports que pourraient avoir les contrôles de validation des relations entre champs

## Résultats
- les principaux résultats sont regroupés sur un [tableau de bord](https://nbviewer.org/github/loco-philippe/Environmental-Sensing/blob/main/python/Validation/irve/Analyse/IRVE_indicateurs.ipynb) 
- la formalisation d'un modèle de données facilite la compréhension des données et des relations entre champs
- l'outil de contrôle permet d'améliorer significativement la qualité des données par l'identification d'incohérences de relations
- l'identification des incohérences permet de trouver des stratégies de réduction des écarts (dans l'exemple ci-dessous, on passe 36% d'écart à 3,8 %)  
- l'analyse des données permet de (re)construire le modèle de données qui minimise les incohérences
- les incohérences détectées sur le jeu de données IRVE restent faibles (inférieures à 5 % des point de charge documentés - voir chapitre 4)

## Suite à donner
- Mettre à jour, valider et publier le modèle de données IRVE
- Définir les contrôles supplémentaires à intégrer pour toutes nouvelles données ainsi que pour le jeu complet
- Mettre en oeuvre les outils de contrôle

## Evolutions possibles 
- Ajouter dans les guides d'Etalab un guide pour les modèles de données 
- Intégrer dans les schémas de données la propriété "relationship" en cours de validation,
- Définir un indicateur qui mesure l'écart (existant / attendu) des relations entre champs

## Sommaire
*(liens actifs sur jupyter Notebook ou Nbviewer)*
- [1 - modèle de données](#1---modèle-de-données)
- [2 - Initialisation](#2---Initialisation)
- [3 - Séparation des pdc itinerance et hors itinerance](#3---Séparation-des-pdc-itinerance-et-hors-itinerance)
- [4 - Bilan initial intégrité](#4---Bilan-initial-intégrité)
- [5 - Séparation doublons pdc - date de maj](#5---Séparation-doublons-pdc---date-de-maj)
- [6 - Séparation doublons station - date de maj](#6---Séparation-doublons-station---date-de-maj)
- [7 - Synthèse](#7---Synthèse)
- [8 - Exemples d'erreurs résiduelles](#8---Exemples-d\'erreurs-résiduelles)

Ce Notebook peut être consulté sur [nbviewer](http://nbviewer.org/github/loco-philippe/Environmental-Sensing/tree/main/python/Validation/irve/Analyse)

données utilisées : https://www.data.gouv.fr/fr/datasets/fichier-consolide-des-bornes-de-recharge-pour-vehicules-electriques/    
fichier : "*consolidation-etalab-schema-irve-statique-v-2.2.0-2023xxxx.csv*"

-----------------------
# 1 - modèle de données

Le modèle de données proposé ci-dessous est construit sur la base du schéma de données mis à disposition et du contenu du jeu de données.    
Il est à consolider en fonction de l'expertise des concepteurs et réutilisateurs (voir [guide méthodologique](https://github.com/loco-philippe/Environmental-Sensing/blob/main/property_relationship/FR_methodology.ipynb)).

*Notation:*
- *M : Mandatory - documentation obligatoire*
- *PK : Primary Key - identifiant unique de l'entité*
- *Root : champ fictif associé à une ligne du tableau*

In [1]:
from base64 import b64encode
from IPython.display import Image, display
with open('IRVE_modele.txt', 'r', encoding="utf-8") as f:
    modele = f.read()
display(Image(url="https://mermaid.ink/img/" + b64encode(modele.encode("ascii")).decode("ascii")))

------
# 2 - Initialisation
## initialisation logicielle

In [2]:
from datetime import datetime
import json
from observation import Sdataset, Analysis
import pandas as pd

def analyse_integrite(data, fields, pr=True):
    '''analyse les relations du DataFrame définies dans le dict fields et retourne un dict avec pour chaque contrôle la
    liste des index ko. Les résultats des contrôles sont également ajoutés sous forme de champs booléens à data'''
    analyse = Analysis(data)
    dic_res = analyse.check_relationship(fields)
    data['ok'] = True
    for name, lis in dic_res.items():
        data[name] = True
        data.loc[lis, name] = False
        data['ok'] = data['ok'] & data[name]
        if pr:
            print('{:<50} {:>5}'.format(name, len(data) - data[name].sum()))
    return dic_res

def add_nbre_pdc(data, name='ok', pr=True):
    '''ajoute un champ avec le nombre de pdc calculés et le compare au champ 'nbre_pdc' '''
    data['nb_pdc_calc'] = data.groupby('id_station_itinerance')['index'].transform('count')
    data['nbre_pdc_ok'] = data['nb_pdc_calc'] == data['nbre_pdc']    
    data['calc-nbre'] = data['nb_pdc_calc'] - data['nbre_pdc']
    #data[name] = data[name] & data['nbre_pdc_ok']
    if pr:
        ecart_nbre = len(data) - data['nbre_pdc_ok'].sum()
        print('{:<20} {:>5}'.format('nbre_pdc_ko', ecart_nbre))
        print(round(ecart_nbre / len(data) * 100), ' %')

## initialisation des données
- lecture du fichier issu de l'api

In [3]:
dates = ['2023-03-03', '2023-04-17', '2023-05-02', '2023-05-24', '2023-06-17', '2023-07-04', '2023-07-23', '2023-08-08']
date= dates[-1]
log = {'date_irve': date, 
    'file': 'consolidation-etalab-schema-irve-statique-v-2.2.0-'+date[:4]+date[5:7]+date[8:]+'.csv',
#    'chemin': 'https://raw.githubusercontent.com/loco-philippe/Environmental-Sensing/gen-ntv/python/Validation/irve/Analyse/'}
#log = {'date_irve': '2022-06-06', 'file': 'consolidation-etalab-schema-irve-v-2.0.2-20220606-propre2.csv',
    'chemin': 'D:\\philippe\\python ESstandard\\Environmental-Sensing\\python\\Validation\\irve\\Analyse\\'}
irve = pd.read_csv(log['chemin'] + log['file'], sep=',', low_memory=False)
log['len_irve'] = len(irve)
print('nombre de lignes : ', log['len_irve']) 

nombre de lignes :  41232


## schéma de données
Le schéma de données restreint à la propriété 'relationship' et construit à partir du modèle de données est le suivants :

In [4]:
# complément à inclure dans le schéma de données
fields = [
 # relation unicité des pdl
 { "name": "index",
   "relationship" : { "parent" : "id_pdc_itinerance", "link" : "coupled" }},   
 # relations inter entités
 { "name": "contact_operateur",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nom_enseigne",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "coordonneesXY",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "id_station_itinerance",
   "relationship" : { "parent" : "id_pdc_itinerance",     "link" : "derived" }},
 # relations intra entité - station
 { "name": "nom_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "implantation_station",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 #{ "name": "date_maj",
 #  "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "nbre_pdc",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "condition_acces",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "horaires",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 { "name": "station_deux_roues",
   "relationship" : { "parent" : "id_station_itinerance", "link" : "derived" }},
 # relations intra entité - localisation
 { "name": "adresse_station",
   "relationship" : { "parent" : "coordonneesXY",         "link" : "derived" }} ]

# liste des champs liés à un controle (relations) et obligatoires (mandatory)
relations = ['index', 'contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 
             'nom_station', 'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 
             'id_pdc_itinerance', 'date_maj', 'last_modified']
mandatory = ['contact_operateur', 'nom_enseigne', 'coordonneesXY', 'adresse_station', 'id_station_itinerance', 'nom_station',
             'implantation_station', 'nbre_pdc', 'condition_acces', 'horaires', 'station_deux_roues', 'id_pdc_itinerance', 
             'puissance_nominale', 'prise_type_ef', 'prise_type_2', 'prise_type_combo_ccs', 'prise_type_chademo', 
             'prise_type_autre', 'paiement_acte', 'paiement_autre', 'reservation',  'accessibilite_pmr', 'restriction_gabarit', 
             'date_maj', 'last_modified']
controles = ['index - id_pdc_itinerance', 'contact_operateur - id_station_itinerance', 'nom_enseigne - id_station_itinerance',
             'coordonneesXY - id_station_itinerance', 'id_station_itinerance - id_pdc_itinerance',
             'nom_station - id_station_itinerance', 'implantation_station - id_station_itinerance',
             'nbre_pdc - id_station_itinerance', 'condition_acces - id_station_itinerance', 'horaires - id_station_itinerance',
             'station_deux_roues - id_station_itinerance', 'adresse_station - coordonneesXY']

-----------------------
## 3 - Séparation des pdc itinerance et hors itinerance
- de l'ordre de 1 % des points de charge sont hors itinerance

In [5]:
# séparation des données
data = irve
data[['id_station_itinerance','id_pdc_itinerance']] = data[['id_station_itinerance','id_pdc_itinerance']].astype('string')
#data[['last_modified','date_maj']] = data[['last_modified','date_maj']].astype('datetime64')
data['non_concerne'] = data['id_station_itinerance'].str.contains('oncern') | data['id_pdc_itinerance'].str.contains('oncern')

non_concerne = data[data['non_concerne']].reset_index()['index']
itinerance = data[~data['non_concerne']].reset_index()
itinerance_init = itinerance.loc[:, relations]
log['pdc_hors_itinerance'] = len(non_concerne)
log['pdc_en_itinerance'] = len(itinerance)
print('nombre de pdc hors itinerance : ', log['pdc_hors_itinerance'])
print('nombre de pdc en itinerance   : ', log['pdc_en_itinerance'])

nombre de pdc hors itinerance :  2318
nombre de pdc en itinerance   :  38914


In [6]:
# arborescence des champs et nombre de valeurs différentes
il = Sdataset(itinerance_init)
print(il.tree())

-1: root-derived (38914)
   0 : index (38914)
      1 : contact_operateur (128)
      2 : nom_enseigne (1007)
      3 : coordonneesXY (12670)
      4 : adresse_station (12176)
      5 : id_station_itinerance (17197)
      6 : nom_station (12775)
      7 : implantation_station (5)
      8 : nbre_pdc (33)
      9 : condition_acces (2)
      10: horaires (299)
      11: station_deux_roues (6)
      12: id_pdc_itinerance (36957)
      13: date_maj (600)
      14: last_modified (494)


-----------------------
## 4 - Bilan initial intégrité
- plus de 50 % des lignes présentent un défaut d'intégrité

In [7]:
# séparation données bonnes (itinerance_ok_1) et données résiduelles (itinerance_1)
res = analyse_integrite(itinerance_init, fields)
itinerance_ok_1 = itinerance_init.loc[itinerance_init.ok, relations].reset_index(drop=True)
itinerance_1 = itinerance_init.loc[~itinerance_init.ok, relations].reset_index(drop=True)
itinerance_init = itinerance_init.loc[:, relations]
log['init_ok'] = len(itinerance_ok_1)
log['init_ko'] = len(itinerance_1)
print("\nnombre d'enregistrements sans erreurs : ", log['init_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['init_ko'])
print("taux d'erreur : ", round(log['init_ko'] / log['pdc_en_itinerance'] * 100), ' %')

index - id_pdc_itinerance                           3868
contact_operateur - id_station_itinerance            997
nom_enseigne - id_station_itinerance                2282
coordonneesXY - id_station_itinerance               1882
id_station_itinerance - id_pdc_itinerance            697
nom_station - id_station_itinerance                 1579
implantation_station - id_station_itinerance        1332
nbre_pdc - id_station_itinerance                     823
condition_acces - id_station_itinerance               31
horaires - id_station_itinerance                      97
station_deux_roues - id_station_itinerance          1409
adresse_station - coordonneesXY                     1102

nombre d'enregistrements sans erreurs :  32534
nombre d'enregistrements avec au moins une erreur :  6380
taux d'erreur :  16  %


-----------------------
## 5 - Séparation doublons pdc - date de maj
- plus de la moitié des pdc en erreur sont liées aux doublons de pdc
- la suppression des doublons permet de réduire de plus de 80% le nombre de lignes erronnées

In [8]:
# séparation doublons pdc (doublons_pdc) et données résiduelles (itinerance_2)
itinerance_1['doublons_pdc'] = itinerance_1.sort_values(by=['date_maj', 'last_modified']).duplicated('id_pdc_itinerance', keep='last')

doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)['index']
#doublons_pdc = itinerance_1[itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_2 = itinerance_1[~itinerance_1['doublons_pdc']].loc[:, relations].reset_index(drop=True)
itinerance_1 = itinerance_1.loc[:, relations]
log['doublons_pdc'] = len(doublons_pdc)
log['sans_doublons_pdc'] = len(itinerance_2)
print('nombre de doublons pdc : ', log['doublons_pdc'], ' soit ', round(log['doublons_pdc']/log['init_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_pdc'])

nombre de doublons pdc :  1957  soit  31  %
nombre de pdc sans doublon   :  4423


In [9]:
# séparation données bonnes (itinerance_ok_3) et données résiduelles (itinerance_3)
res = analyse_integrite(itinerance_2, fields)
itinerance_ok_3 = itinerance_2.loc[itinerance_2.ok, relations].reset_index(drop=True)
itinerance_3 = itinerance_2.loc[~itinerance_2.ok, relations].reset_index(drop=True)
itinerance_2 = itinerance_2.loc[:, relations]
log['etape3_ok'] = len(itinerance_ok_3)
log['etape3_ko'] = len(itinerance_3)
print("\nnombre d'enregistrements sans erreurs : ", log['etape3_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape3_ko'])

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance             67
nom_enseigne - id_station_itinerance                 337
coordonneesXY - id_station_itinerance                920
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  477
implantation_station - id_station_itinerance         287
nbre_pdc - id_station_itinerance                     789
condition_acces - id_station_itinerance               13
horaires - id_station_itinerance                      56
station_deux_roues - id_station_itinerance           513
adresse_station - coordonneesXY                      356

nombre d'enregistrements sans erreurs :  1903
nombre d'enregistrements avec au moins une erreur :  2520


-----------------------
## 6 - Séparation doublons station - date de maj
- plus de 15% des erreurs résiduelles sont liées au mélange d'anciens et de nouveaux pdc
- la suppression des anciens pdc permet de réduire de plus de 20% le nombre de lignes erronnées
- les dernières erreurs correspondent à des stations associées à 33 opérateurs et sont liées à des causes multiples

In [10]:
# séparation doublons stations (doublons_stat_maj) et données résiduelles (itinerance_4)
itinerance_3['stat_maj'] = itinerance_3.id_station_itinerance + itinerance_3.date_maj
stat_maj_unique = itinerance_3.sort_values(by='stat_maj').drop_duplicates('id_station_itinerance', keep='last')
itinerance_3['last_stat_maj'] = itinerance_3['stat_maj'].isin(stat_maj_unique['stat_maj'])

doublons_stat_maj = itinerance_3[~itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)['index']
itinerance_4 = itinerance_3[itinerance_3['last_stat_maj']].loc[:, relations].reset_index(drop=True)
itinerance_3 = itinerance_3.loc[:, relations]
log['doublons_station'] = len(doublons_stat_maj)
log['sans_doublons_station'] = len(itinerance_4)
print('nombre de doublons stations : ', log['doublons_station'], ' soit ', 
      round(log['doublons_station']/log['etape3_ko'] * 100), ' %')
print('nombre de pdc sans doublon   : ', log['sans_doublons_station'])

nombre de doublons stations :  479  soit  19  %
nombre de pdc sans doublon   :  2041


In [11]:
# séparation données bonnes (itinerance_ok_5) et données résiduelles (itinerance_5 / itinerance_5_full)
res = analyse_integrite(itinerance_4, fields)
itinerance_ok_5 = itinerance_4.loc[itinerance_4.ok, relations].reset_index(drop=True)
itinerance_5_full = itinerance_4.loc[~itinerance_4.ok].reset_index(drop=True)
itinerance_5 = itinerance_5_full.loc[:, relations]
#itinerance_4 = itinerance_4.loc[:, relations]
log['etape5_ok'] = len(itinerance_ok_5)
log['etape5_ko'] = len(itinerance_5)
print("\nnombre d'enregistrements sans erreurs : ", log['etape5_ok'])
print("nombre d'enregistrements avec au moins une erreur : ", log['etape5_ko'], ' soit ', 
      round(log['etape5_ko']/log['etape3_ko'] * 100), ' %')

index - id_pdc_itinerance                              0
contact_operateur - id_station_itinerance              2
nom_enseigne - id_station_itinerance                 263
coordonneesXY - id_station_itinerance                377
id_station_itinerance - id_pdc_itinerance              0
nom_station - id_station_itinerance                  195
implantation_station - id_station_itinerance          52
nbre_pdc - id_station_itinerance                     539
condition_acces - id_station_itinerance                8
horaires - id_station_itinerance                      52
station_deux_roues - id_station_itinerance           448
adresse_station - coordonneesXY                      319

nombre d'enregistrements sans erreurs :  99
nombre d'enregistrements avec au moins une erreur :  1942  soit  77  %


In [12]:
# structure des données résiduelles et nombre de valeurs
il = Sdataset(itinerance_5)
print(il.tree())

-1: root-derived (1942)
   0 : index (1942)
      1 : contact_operateur (31)
      2 : nom_enseigne (338)
      3 : coordonneesXY (479)
      4 : adresse_station (515)
         14: last_modified (60)
      5 : id_station_itinerance (523)
         13: date_maj (75)
      6 : nom_station (472)
      7 : implantation_station (5)
      8 : nbre_pdc (17)
      9 : condition_acces (2)
      10: horaires (69)
      11: station_deux_roues (5)
      12: id_pdc_itinerance (1942)


In [13]:
# nb maxi d'erreurs
itinerance_4['somme'] = 0
for name in res.keys():
    itinerance_4['somme'] += 1 - itinerance_4[name]
erreurs = max(itinerance_4['somme'])
maxi = itinerance_4[itinerance_4.somme >= erreurs]
print("nombre d'enregistrements avec ", erreurs, " erreurs : ", len(maxi))

nombre d'enregistrements avec  4  erreurs :  4


-----------------------
## 7 - Synthèse


### fichiers
Génération des fichiers intégrant les défauts d'intégrité :
- fichier csv des lignes résiduelles à traiter (IRVE_itinerance_residuel)
- fichier csv des données itinerance avec indicateur des données à corriger ou à ignorer (IRVE_itinerance_complet)
- fichier csv des données itinerance valides (IRVE_itinerance_valide)
- fichier csv des doublons (IRVE_itinerance_doublons)

In [14]:
# consolidation des données
itinerance['doublons_stat_maj'] = itinerance['index'].isin(doublons_stat_maj)
itinerance['doublons_pdc'] = itinerance['index'].isin(doublons_pdc)
itinerance['lignes_a_corriger'] = itinerance['index'].isin(itinerance_5['index'])
itinerance['doublons_a_supprimer'] = itinerance['doublons_stat_maj'] | itinerance['doublons_pdc']
itinerance['lignes_ko'] = itinerance['doublons_a_supprimer'] | itinerance['lignes_a_corriger']
print('total des lignes à corriger : ', itinerance['lignes_a_corriger'].sum())
itinerance_doublons = itinerance[itinerance['doublons_a_supprimer']].reset_index(drop=True)
print('total des doublons à supprimer : ', len(itinerance_doublons))
itinerance_ok = itinerance[~itinerance['lignes_ko']].reset_index(drop=True)
print('nombre de pdc avec controles ok : ', len(itinerance_ok))

total des lignes à corriger :  1942
total des doublons à supprimer :  2436
nombre de pdc avec controles ok :  34536


In [15]:
#génération des fichiers
extension = log['date_irve'] +'.csv'
itinerance_5_full.to_csv('IRVE_itinerance_residuel' + extension)
itinerance.to_csv('IRVE_itinerance_complet' + extension)
itinerance_ok.to_csv('IRVE_itinerance_valide' + extension)
itinerance_doublons.to_csv('IRVE_itinerance_doublons' + extension)
log['IRVE_itinerance_residuel' + extension] = len(itinerance_5_full)
log['IRVE_itinerance_complet' + extension] = len(itinerance)
log['IRVE_itinerance_valide' + extension] = len(itinerance_ok)
log['IRVE_itinerance_valide_stat' + extension] = len(itinerance_ok.drop_duplicates('id_station_itinerance', keep='last'))
log['IRVE_itinerance_doublons' + extension] = len(itinerance_doublons)

### vérification de l'intégrité

In [16]:
# vérification de l'absence d'erreurs
res = analyse_integrite(itinerance_ok.loc[:, relations], fields, pr=False)
log['bilan_erreurs'] = sum([len(controle) for controle in res.values()])
log['date'] = datetime.now().isoformat()
print('bilan intégrité :')
print('    erreurs : ', log['bilan_erreurs'])

bilan intégrité :
    erreurs :  0


In [17]:
# structure des données bonnes
il = Sdataset(itinerance_ok.loc[:, mandatory])
print(il.tree())

-1: root-derived (34536)
   11: id_pdc_itinerance (34536)
      4 : id_station_itinerance (16491)
         1 : nom_enseigne (675)
         2 : coordonneesXY (11705)
            0 : contact_operateur (111)
            3 : adresse_station (11088)
            8 : condition_acces (2)
         5 : nom_station (11807)
         6 : implantation_station (5)
         7 : nbre_pdc (33)
         9 : horaires (235)
         10: station_deux_roues (5)
         19: paiement_autre (6)
      12: puissance_nominale (72)
      13: prise_type_ef (8)
      14: prise_type_2 (7)
      15: prise_type_combo_ccs (7)
      16: prise_type_chademo (7)
      17: prise_type_autre (6)
      18: paiement_acte (8)
      20: reservation (6)
      21: accessibilite_pmr (4)
      22: restriction_gabarit (108)
      23: date_maj (437)
      24: last_modified (407)


### Indicateurs

In [18]:
# génération des indicateurs liés aux règles
regles = ['Pdc non unique', 'Station multi-operateurs', 'Station multi-enseignes', 'Station multi-localisations', 
          'Pdc multi-stations', 'station avec plusieurs noms', 'station multi-implantations', 
          'nombre de pdc par station incoherent', 'station multi-acces', 'station multi-horaires', 
          'acces deux-roues incoherent', 'localisation multi-adresses']
principal = [16, 17, 18, 19, 20]
secondaire = [21, 22, 23, 24, 25, 26, 27]
irve = itinerance_5_full # residuel
total = len(irve)
indic = {}

irve['principal'] = True
for ind in principal:
    irve['principal'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] = True
for ind in secondaire:
    irve['secondaire'] &= irve.iloc[:,ind]
    indic[regles[ind-16]] = int(total - irve.iloc[:,ind].sum())
irve['secondaire'] |= (~irve['principal'] & ~irve['secondaire'])
irve['verif'] = irve['principal'] & irve['secondaire']
indic['principal pdc'] = int(total - irve['principal'].sum())
indic['secondaire pdc'] = int(total - irve['secondaire'].sum())

irve_p = irve[~irve['principal']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
irve_s = irve[~irve['secondaire']].drop_duplicates('id_station_itinerance').reset_index(drop=True)
indic['principal stat'] = len(irve_p)
indic['secondaire stat'] = len(irve_s)



In [19]:
# ajout des principaux opérateurs contributeur de défauts
operateurs = list(itinerance_5.drop_duplicates('contact_operateur')['contact_operateur'])
erreurs_op = [len(itinerance_5.loc[itinerance_5.contact_operateur == operateur]) for operateur in operateurs]
err_op, oper = tuple(zip(*(sorted(zip(erreurs_op, operateurs), reverse=True))[:5]))
indic['operateurs'] = list(oper)
indic['erreurs_operateurs'] = list(err_op)

print(len(err_op), ' opérateurs représentent : ', round(sum(err_op) / sum(erreurs_op) * 100), ' % des écarts\n')
for err, op in zip(indic['erreurs_operateurs'], indic['operateurs']):
    print(op, err)

5  opérateurs représentent :  76  % des écarts

supervision-ev.france@totalenergies.com 506
assistance-commerciale@e-vadea.fr 321
pascal.lhermitte@siege27.fr 260
web@freshmile.com 207
contact@e55c.com 184


In [20]:
# ajout de l'indicateur d'écart entre nombre de pdc calculés et saisis
add_nbre_pdc(irve, pr=True)
indic['nb_pdc_calc-nbre_pdc'] = int(irve['calc-nbre'].sum())

nbre_pdc_ko            876
45  %


In [21]:
# stockage des indicateurs
log |= indic
with open('logfile.txt', 'a', encoding="utf-8") as f:
    f.write(json.dumps(log) + '\n')

-----------------------
## 8 - Exemples d'erreurs résiduelles 

### Erreurs multiples 
- exemple avec le nombre maximal d'erreurs (4) (2 stations soit 29 pdc avec : adresse, nom, nbre_pdc et deux-roues erronés)

In [22]:
maxi.loc[:, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1493,34784,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299351,2022-11-03,2023-08-08T07:16:44.947000+00:00
1494,34785,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",AVENUE DE PARIS VERSAILLES,FR55CP78000,AVENUE DE PARIS VERSAILLES,Voirie,4,Accès réservé,24/7,FALSE,FR55CE780004880188021299331,2022-11-03,2023-08-08T07:16:44.947000+00:00
1689,35234,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S0,2022-11-03,2023-08-08T07:16:44.947000+00:00
1690,35235,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S1,2022-11-03,2023-08-08T07:16:44.947000+00:00


### Cohérence implantation_station - id_station
- 64 pdc sont liés à une erreur de choix d'implantation (3 stations, 1 opérateur)

In [23]:
itinerance_4.loc[~itinerance_4['implantation_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
45,5970,info@allego.eu,Geant Casino Aix en Provence,"[5.41683791, 43.53314177]","Avenue de Brédasque, 13097 Aix-en-Provence",FRALLEGO002046,Geant Casino Aix en Provence,Station dédiée à la recharge rapide,3,Accès libre,24/7,false,FRALLEGO0020463,2023-07-09,2023-08-08T19:05:52.847000+00:00
46,5971,info@allego.eu,Geant Casino Aix en Provence,"[5.41683791, 43.53314177]","Avenue de Brédasque, 13097 Aix-en-Provence",FRALLEGO002046,Geant Casino Aix en Provence,Station dédiée à la recharge rapide,3,Accès libre,24/7,false,FRALLEGO0020462,2023-07-09,2023-08-08T19:05:52.847000+00:00
47,5972,info@allego.eu,Geant Casino Aix en Provence,"[5.41683791, 43.53314177]","Avenue de Brédasque, 13097 Aix-en-Provence",FRALLEGO002046,Geant Casino Aix en Provence,Voirie,3,Accès libre,24/7,false,FRALLEGO0020461,2023-07-09,2023-08-08T19:05:52.847000+00:00
48,5974,info@allego.eu,Geant Casino Saint Louis,"[7.557195, 47.583132]","45 Avenue Général de Gaulle, 68300 Saint-Louis",FRFR1EFRALLEGO002007,Geant Casino Saint Louis,Station dédiée à la recharge rapide,9,Accès libre,24/7,false,FRFR1EFRALLEGO0020071,2023-07-21,2023-08-08T19:05:52.847000+00:00
49,5975,info@allego.eu,Geant Casino Saint Louis,"[7.557195, 47.583132]","45 Avenue Général de Gaulle, 68300 Saint-Louis",FRFR1EFRALLEGO002007,Geant Casino Saint Louis,Station dédiée à la recharge rapide,9,Accès libre,24/7,false,FRFR1EFRALLEGO0020072,2023-07-21,2023-08-08T19:05:52.847000+00:00
50,5976,info@allego.eu,Geant Casino Saint Louis,"[7.557195, 47.583132]","45 Avenue Général de Gaulle, 68300 Saint-Louis",FRFR1EFRALLEGO002007,Geant Casino Saint Louis,Voirie,9,Accès libre,24/7,false,FRFR1EFRALLEGO0020073,2023-07-21,2023-08-08T19:05:52.847000+00:00
51,6056,info@allego.eu,Allego Geant Casino Montpellier,"[3.889034, 43.58858]","504 Avenue du Mas d'Argelliers, 34000 Montpellier",FRALLEGO002051,Allego Geant Casino Montpellier,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRALLEGO0020512,2023-07-22,2023-08-08T19:05:52.847000+00:00
52,6057,info@allego.eu,Allego Geant Casino Montpellier,"[3.889034, 43.58858]","504 Avenue du Mas d'Argelliers, 34000 Montpellier",FRALLEGO002051,Allego Geant Casino Montpellier,Voirie,8,Accès libre,24/7,false,FRALLEGO0020511,2023-07-22,2023-08-08T19:05:52.847000+00:00
53,6558,info@allego.eu,Allego Geant Casino Avignon,"[4.83458514, 43.9278626]","162 Avenue Pierre Semard, 84000 Avignon",FRALLEGO002093,Allego Geant Casino Avignon,Voirie,8,Accès libre,24/7,false,FRALLEGO0020931,2023-07-21,2023-08-08T19:05:52.847000+00:00
54,6563,info@allego.eu,Allego Geant Casino Avignon,"[4.83458514, 43.9278626]","162 Avenue Pierre Semard, 84000 Avignon",FRALLEGO002094,Allego Geant Casino Avignon,Station dédiée à la recharge rapide,8,Accès libre,24/7,false,FRALLEGO0020942,2023-07-21,2023-08-08T19:05:52.847000+00:00


### Cohérence nom_station - id_station
- 328 pdc sont associés à une station avec un nom non cohérent

In [24]:
itinerance_4.loc[~itinerance_4['nom_station - id_station_itinerance'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
771,22610,bo_dgic@seolis.net,AlterBase,"[-0.4604029,46.3288736]",Place Chanzy - 79000 NIORT,FRSEOPAB79191P0031A,Réseau AlterBase - Niort - Conseil Département...,Parking public,2,Accès libre,24/7,false,FRSEOEAB79191P0031A11,2023-07-25,2023-08-08T07:18:59.858000+00:00
772,22611,bo_dgic@seolis.net,AlterBase,"[-0.4604029,46.3288736]",Place Chanzy - 79000 NIORT,FRSEOPAB79191P0031A,Réseau AlterBase - Niort - Conseil Département...,Parking public,2,Accès libre,24/7,false,FRSEOEAB79191P0031A12,2023-07-25,2023-08-08T07:18:59.858000+00:00
1347,29333,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM021,2023-06-02,2023-08-08T07:17:25.387000+00:00
1348,29334,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM020,2023-06-02,2023-08-08T07:17:25.387000+00:00
1349,29335,supervision-ev.france@totalenergies.com,INTERPARKING,"[7.265881, 43.695113]",1 Promenade des anglais 6000 NICE,FRIPKNRME,INTERPARKING - Nice Ruhl Méridien,Parking privé à usage public,22,Accès libre,24/7,false,FRIPKENRM019,2023-06-02,2023-08-08T07:17:25.387000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1690,35235,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.130358, 48.801996]",6 AVENUE DE PARIS -VERSAILLES,FR55CP78000,6 AVENUE DE PARIS -VERSAILLES,Voirie,2,Accès réservé,24/7,FALSE,FR55CEFR78000AVDEPAR1S1,2022-11-03,2023-08-08T07:16:44.947000+00:00
1878,36740,pascal.lhermitte@siege27.fr,SIEGE50,"[1.221452, 49.197647]",Ecoparc satellite ZA1 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900012,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC1G,2021-06-15,2023-08-08T07:16:29.698000+00:00
1892,36754,pascal.lhermitte@siege27.fr,SIEGE49,"[1.221452, 49.197647]",Ecoparc satellite ZA1 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900012,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC1D,2021-06-15,2023-08-08T07:16:29.698000+00:00
1952,36815,pascal.lhermitte@siege27.fr,SIEGE87,"[1.224898, 49.195263]",Ecoparc satellite ZA2 Heudebouville,FRS27PHEUDEBOUVILLEECOPARC,900031,Voirie,2,Accès libre,24/7,false,FRS27EHEUDEBOUVILLECOPARC2D,2021-06-15,2023-08-08T07:16:29.698000+00:00


### Cohérence adresse - coordonnées
- 440 pdc ont une adresse non cohérente avec les coordonnées géographiques

In [25]:
itinerance_4.loc[~itinerance_4['adresse_station - coordonneesXY'], relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
0,1354,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34212,SIGEIF - 4 RUE CRISTINO - GARCIA PLACE ETIENNE...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34212,2023-08-09,2023-08-09T02:54:38.115000+00:00
1,1355,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA PLACE ETIENNE DOLET,FRSIGPSIGE34211,SIGEIF - 4 RUE CRISTINO - GARCIA PLACE ETIENNE...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34211,2023-08-09,2023-08-09T02:54:38.115000+00:00
2,1358,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34411,SIGEIF - 4 RUE CRISTINO GARCIA - PLACE ETIENNE...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34411,2023-08-09,2023-08-09T02:54:38.115000+00:00
3,1360,sav@izivia.com,SIGEIF,"[2.27736, 48.9917]",RUE CRISTINO GARCIA,FRSIGPSIGE34412,SIGEIF - 4 RUE CRISTINO GARCIA - PLACE ETIENNE...,Voirie,2,Accès libre,24/7,false,FRSIGESIGE34412,2023-08-09,2023-08-09T02:54:38.115000+00:00
4,2981,sav@izivia.com,SOREGIES,"[0.30007, 46.54829]",RUE DE LÉCORCERIE,FRS86PSOREB1511,POITIERS SUD HPC,Voirie,1,Accès libre,24/7,false,FRS86ESOREB1511,2023-08-09,2023-08-09T02:54:34.407000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,37573,contact@ubitricity.com,Ubitricity France Network,"[0.223944, 49.564167]",Rue de Chemin de Fer - Epouville,FRUBIE10062883,Rue de Chemin de Fer - Epouville,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10062883,2023-07-03,2023-08-08T07:15:08.299000+00:00
2008,37658,contact@ubitricity.com,Ubitricity France Network,"[0.073833, 49.510417]",30 Rue Georges Boissaye du Bocage - Sainte-Adr...,FRUBIE10099360,30 Rue Georges Boissaye du Bocage - Sainte-Adr...,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10099360,2023-05-10,2023-08-08T07:14:58.458000+00:00
2009,37659,contact@ubitricity.com,Ubitricity France Network,"[0.073833, 49.510417]",30 Rue Georges Boissaye du Bocage - Sainte-Adr...,FRUBIE10095460,30 Rue Georges Boissaye du Bocage - Sainte-Adr...,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10095460,2023-05-10,2023-08-08T07:14:58.458000+00:00
2010,37660,contact@ubitricity.com,Ubitricity France Network,"[0.223944, 49.564167]",Rue du Chemin de Fer - Épouville,FRUBIE10062826,Rue du Chemin de Fer - Épouville,Parking public,1,Accès libre,24/7,FALSE,FRUBIE10062826,2023-05-10,2023-08-08T07:14:58.458000+00:00


- exemple : une station avec plusieurs adresses -> erreur de saisie ?

In [26]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[2.87930851314442, 48.94679007929618]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1239,28455,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003958,2023-01-27,2023-08-08T07:17:25.387000+00:00
1240,28456,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",51 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003955,2023-01-27,2023-08-08T07:17:25.387000+00:00
1241,28457,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",50 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003954,2023-01-27,2023-08-08T07:17:25.387000+00:00
1242,28469,supervision-ev.france@totalenergies.com,RIESTER MEAUX,"[2.87930851314442, 48.94679007929618]",52 Rue François De Tessan 77100 Meaux,FRTCBP00702,RIESTER MEAUX,Parking privé à usage public,4,Accès libre,"Mo-Fr 08:30-12:00,Mo-Fr 13:30-19:00,Sa 09:00-1...",false,FRTCBE003959,2023-01-27,2023-08-08T07:17:25.387000+00:00


### Cohérence station - coordonnées
- 477 pdc sont associés à des stations avec plusieurs coordonnées

In [27]:
itinerance_4.loc[~itinerance_4['coordonneesXY - id_station_itinerance'], relations][200:215]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1507,34811,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.585713, 43.230239]",OFFICE DU TOURISME - GASSIN,FR55CP83580,OFFICE DU TOURISME - GASSIN,Voirie,2,Accès réservé,24/7,FALSE,FR55CE835804323023965857121,2022-11-03,2023-08-08T07:16:44.947000+00:00
1508,34812,contact@e55c.com,ELECTRIC 55 CHARGING,"[6.585713, 43.230239]",OFFICE DU TOURISME - GASSIN,FR55CP83580,OFFICE DU TOURISME - GASSIN,Voirie,2,Accès réservé,24/7,FALSE,FR55CE835804323023965857111,2022-11-03,2023-08-08T07:16:44.947000+00:00
1513,34820,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553361,2022-11-03,2023-08-08T07:16:44.947000+00:00
1514,34821,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553331,2022-11-03,2023-08-08T07:16:44.947000+00:00
1515,34822,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553341,2022-11-03,2023-08-08T07:16:44.947000+00:00
1516,34823,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553351,2022-11-03,2023-08-08T07:16:44.947000+00:00
1517,34824,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553311,2022-11-03,2023-08-08T07:16:44.947000+00:00
1518,34825,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.262841, 48.798985]",PLACE FERRARI - CLAMART,FR55CP92140,PLACE FERRARI - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404878689922553321,2022-11-03,2023-08-08T07:16:44.947000+00:00
1519,34826,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628421,2022-11-03,2023-08-08T07:16:44.947000+00:00
1520,34827,contact@e55c.com,ELECTRIC 55 CHARGING,"[2.255335, 48.786899]",HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,FR55CP92140,HOPITAL BECLERE - AVENUE DE GAULLE - CLAMART,Voirie,6,Accès réservé,24/7,FALSE,FR55CE921404879898422628411,2022-11-03,2023-08-08T07:16:44.947000+00:00


- exemple : une station avec plusieurs coordonnées -> incompréhension distinction station / pdc

In [28]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR073PCAMAIEUFR', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1924,39670,acelec@acelec-france.com,Camaïeu France,"[3.207187,50.684964]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ETCIX5105,2022-07-21,2023-08-08T07:14:16.344000+00:00
1925,39672,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EREUO5946,2022-07-21,2023-08-08T07:14:16.344000+00:00
1926,39673,acelec@acelec-france.com,Camaïeu France,"[3.207433,50.684876]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EQ1GI51176,2022-07-21,2023-08-08T07:14:16.344000+00:00
1927,39674,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073ECC1C5530,2022-07-21,2023-08-08T07:14:16.344000+00:00
1928,39675,acelec@acelec-france.com,Camaïeu France,"[3.20737,50.684898]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EO0AV51091,2022-07-21,2023-08-08T07:14:16.344000+00:00
1929,39676,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EK0BJ522,2022-07-21,2023-08-08T07:14:16.344000+00:00
1930,39677,acelec@acelec-france.com,Camaïeu France,"[3.207306,50.684918]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E0HKH51125,2022-07-21,2023-08-08T07:14:16.344000+00:00
1931,39678,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073E8OV45243,2022-07-21,2023-08-08T07:14:16.344000+00:00
1932,39679,acelec@acelec-france.com,Camaïeu France,"[3.207246,50.68494]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EYLF65474,2022-07-21,2023-08-08T07:14:16.344000+00:00
1933,39680,acelec@acelec-france.com,Camaïeu France,"[3.207124,50.684984]",211 Av. Jules Brame 59100 Roubaix,FR073PCAMAIEUFR,CAMAÏEU FRANCE,Parking privé à usage public,1,Accès libre,Mo-Su 08:00-08:00,false,FR073EBEQA5957,2022-07-21,2023-08-08T07:14:16.344000+00:00


- exemple : une station avec plusieurs coordonnées -> regroupement de stations ?

In [29]:
itinerance_5.loc[itinerance_5.id_station_itinerance == 'FR55CPBP514', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
1450,34855,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A1,2022-11-03,2023-08-08T07:16:44.947000+00:00
1451,34856,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A2,2022-11-03,2023-08-08T07:16:44.947000+00:00
1452,34857,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242799, 49.120583]",BP CHAMPAGNE SUD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE SUD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02AA8A3,2022-11-03,2023-08-08T07:16:44.947000+00:00
1453,34858,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6A61,2022-11-03,2023-08-08T07:16:44.947000+00:00
1454,34859,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B62,2022-11-03,2023-08-08T07:16:44.947000+00:00
1455,34860,contact@e55c.com,ELECTRIC 55 CHARGING,"[4.242644, 49.120556]",BP CHAMPAGNE NORD - LES PETITES-LOGES,FR55CPBP514,BP CHAMPAGNE NORD - LES PETITES-LOGES - 69800,Station dédiée à la recharge rapide,3,Accès réservé,24/7,FALSE,FR55CE51400GIRVATMX02A6B63,2022-11-03,2023-08-08T07:16:44.947000+00:00


- exemple : quatre stations avec une localisation identique mais deux adresses différentes + id identique entre station et pdc -> erreur d'adresse et erreur d'id_station ?

In [30]:
itinerance_5.loc[itinerance_5.coordonneesXY	 == '[-0.366184, 43.332508]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
37,5066,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649712,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649712,2023-08-09,2023-08-09T02:54:29.511000+00:00
38,5076,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 1,FRS64PMB649711,PAU - PARKING ZENITH 1,Voirie,2,Accès libre,24/7,false,FRS64EMB649711,2023-08-09,2023-08-09T02:54:29.511000+00:00
39,5077,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649812,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649812,2023-08-09,2023-08-09T02:54:29.511000+00:00
40,5079,sav@izivia.com,MOBIVE - SDEPA 64,"[-0.366184, 43.332508]",PARKING ZÉNITH 2,FRS64PMB649811,PAU - PARKING ZENITH 2,Voirie,2,Accès libre,24/7,false,FRS64EMB649811,2023-08-09,2023-08-09T02:54:29.511000+00:00


### Incohérence 'nbre_pdc'
- 25% des pdc ont un champ 'nbre_pdc' mal documenté

- exemple : station avec un pdc 'de regroupement' qui évite de documenter plusieurs lignes

In [31]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRLUMEACACIAS11', 
               ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
30412,FRLUMEACACIAS11,FRLUMEACACIAS11,"[3.1485441,50.7582304]",4


- exemple : station avec plusieurs pdc mais l'id_station est identique à l'id_pdc

In [32]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[2.451322, 45.66523]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
5920,FRSDGEMBDG512,FRSDGPMBDG512,"[2.451322, 45.66523]",2
5921,FRSDGEMBDG511,FRSDGPMBDG511,"[2.451322, 45.66523]",2


- exemple : station avec plusieurs pdc mais le champs a toujours une valeur de 1

In [33]:
itinerance_init.loc[itinerance_init.coordonneesXY	 == '[-1.7548354193520386, 48.125067030488154]', 
                   ['id_pdc_itinerance', 'id_station_itinerance', 'coordonneesXY', 'nbre_pdc']]

,id_pdc_itinerance,id_station_itinerance,coordonneesXY,nbre_pdc
36783,FRIENE35353A51,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
36784,FRIENE35353A11,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
36785,FRIENE35353A21,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
36786,FRIENE35353A41,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1
36787,FRIENE35353A31,FRIENS35353A,"[-1.7548354193520386, 48.125067030488154]",1


### cohérence station - enseigne
- exemple de station avec plusieurs noms d'enseigne

In [34]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS27PBARREOUCHEMOMORT', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
36185,36643,pascal.lhermitte@siege27.fr,SIEGE266,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTG,2021-06-15,2023-08-08T07:16:29.698000+00:00
36187,36645,pascal.lhermitte@siege27.fr,SIEGE265,"[0.66299, 48.946159]",Place du monument aux Morts La Barre en Ouche,FRS27PBARREOUCHEMOMORT,900120,Voirie,2,Accès libre,24/7,false,FRS27EBARREOUCHEMOMORTD,2021-06-15,2023-08-08T07:16:29.698000+00:00


### cohérence station - horaires
- exemple de station avec plusieurs types d'horaire

In [35]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRS23D2302001', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
36690,37162,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:01,False,FRS23P230200101,2023-03-09,2023-08-08T07:16:16.268000+00:00
36691,37163,contact@mobive.fr,Réseau de recharge Creuse,"[2.297630254367735, 45.98220366857867]",Place du Treix,FRS23D2302001,BELLEGARDE EN MARCHE,Parking public,2,Accès libre,Mo-Su 00:00-00:02,False,FRS23P230200102,2023-03-09,2023-08-08T07:16:16.268000+00:00


### cohérence station - deux-roues
- exemple de station avec plusieurs indications d'accessibilité deux-roues

In [36]:
itinerance_init.loc[itinerance_init.id_station_itinerance == 'FRV75PPX1902', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
27051,27473,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19027,2023-07-06,2023-08-08T07:17:25.957000+00:00
27071,27493,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19026,2023-07-06,2023-08-08T07:17:25.957000+00:00
27072,27494,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19025,2023-07-06,2023-08-08T07:17:25.957000+00:00
27073,27495,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19024,2023-07-06,2023-08-08T07:17:25.957000+00:00
27074,27496,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19023,2023-07-06,2023-08-08T07:17:25.957000+00:00
27092,27514,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,False,FRV75EPX19021,2023-07-06,2023-08-08T07:17:25.957000+00:00
27093,27515,supervision-ev.france@totalenergies.com,Belib',"[2.3730799, 48.888195]",61 Avenue de Flandre 75019 Paris,FRV75PPX1902,Paris | Avenue de Flandre 61,Voirie,7,Accès libre,24/7,True,FRV75EPX19022,2023-07-06,2023-08-08T07:17:25.957000+00:00


### cohérence coordonnées - adresse
- exemple de plusieurs stations avec des coordonnées identiques mais des adresses différentes

In [37]:
itinerance_5.loc[itinerance_5.coordonneesXY == '[2.460441, 50.78763]', relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified


### données d'un opérateur
- exemple Virta

In [38]:
itinerance_operateur = itinerance_5_full.loc[itinerance_5_full.contact_operateur == 'support@virta.global']
#itinerance_operateur.loc[itinerance_operateur['adresse_station - coordonneesXY'] == False, relations]
itinerance_operateur.loc[itinerance_operateur['nom_station - id_station_itinerance'] == False, relations]

,index,contact_operateur,nom_enseigne,coordonneesXY,adresse_station,id_station_itinerance,nom_station,implantation_station,nbre_pdc,condition_acces,horaires,station_deux_roues,id_pdc_itinerance,date_maj,last_modified
